In [1]:
train_path = 'fv2_train.parquet'
BATCH_SIZE = 16384 
NUM_CROSS_LAYERS = 3
LR = 0.001
EPOCHS = 4
model_path = '2.1_DCN_MLP.pth'

In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
import pandas as pd
import torch.nn as nn
import torch.optim as optim

In [3]:
torch.manual_seed(42)  
torch.cuda.manual_seed_all(42)  
np.random.seed(42)  

In [4]:
custom_data_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/custom_data/'

train = pd.read_parquet(f'{custom_data_folder}{train_path}', engine='pyarrow')
train['target'] = train['target'].replace({-1:0, 0:1, 1:2})

data_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/data/'

items_meta = pd.read_parquet(f'{data_folder}items_meta.parquet', engine='pyarrow')
items_meta['item_id'] = items_meta['item_id'].astype('category')
items_meta['source_id'] = items_meta['source_id'].astype('category')
items_meta.set_index('item_id', inplace=True)

# Преобразуем embeddings в словарь
item_embeddings_dict = items_meta['embeddings'].to_dict()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class MyDataset(Dataset):
    def __init__(self, interactions, device):
        
        self.device = device
        self.users = torch.tensor(interactions['user_id'].values, dtype=torch.long, device=self.device)
        self.items = torch.tensor(interactions['item_id'].values, dtype=torch.long, device=self.device)
        self.ages = torch.tensor(interactions['age'].values, dtype=torch.long, device=self.device)
        self.item_durations = torch.tensor(interactions['item_duration'].values, dtype=torch.long, device=self.device)
        
        # Проверяем наличие target в данных
        if 'target' in interactions.columns:
            self.targets = torch.tensor(interactions['target'].values, dtype=torch.float32, device=self.device)
        else:
            self.targets = None  # Для тестовых данных target может отсутствовать

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        if self.targets is not None:
            return self.users[idx], self.items[idx], self.ages[idx], self.item_durations[idx], self.targets[idx]
        else:
            return self.users[idx], self.items[idx], self.ages[idx], self.item_durations[idx]


In [6]:
# Создаем экземпляры для обучения и валидации
train_ds = MyDataset(train, device)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE)

In [7]:
class DCN(nn.Module):
    def __init__(self, input_dim, num_cross_layers):
        super(DCN, self).__init__()
        self.input_dim = input_dim
        self.num_cross_layers = num_cross_layers
        
        # Параметры для слоев пересечения
        self.cross_weights = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim, 1)) for _ in range(num_cross_layers)]
        )
        self.cross_biases = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim)) for _ in range(num_cross_layers)]
        )
        
    def forward(self, x):
        # Инициализируем x0
        x0 = x
        for i in range(self.num_cross_layers):
            x = x0 * (x @ self.cross_weights[i]) + self.cross_biases[i] + x
        return x

class DCNWithMLP(nn.Module):
    def __init__(self, input_dim, num_cross_layers=3, hidden_dim=128, output_dim=3):
        super(DCNWithMLP, self).__init__()
        
        # Нормализация входных данных
        self.batch_norm = nn.BatchNorm1d(input_dim)
        
        # DCN модуль
        self.dcn = DCN(input_dim, num_cross_layers)
        
        # MLP модуль
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 64)
        self.fc3 = nn.Linear(64, output_dim)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # Применяем нормализацию входных данных
        x = self.batch_norm(x)
        
        # Пропускаем через DCN
        x = self.dcn(x)
        
        # Пропускаем через MLP
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Определяем параметры
input_dim = len(train.columns) - 1 + 32
num_cross_layers = NUM_CROSS_LAYERS  # Количество слоев DCN

# Создаем модель
model = DCNWithMLP(input_dim, num_cross_layers).to(device)

In [8]:
# Кросс-энтропийная функция потерь для многоклассовой классификации
criterion = nn.CrossEntropyLoss()

# Оптимизатор
optimizer = optim.Adam(model.parameters(), lr=LR)

In [9]:
# Преобразуем embeddings в массив и храним в tensor
item_embeddings_array = torch.tensor(
    np.stack(items_meta['embeddings'].values), 
    device=device, 
    dtype=torch.float32
)

# Сохраняем индексы для быстрого доступа
item_id_to_index = {item: idx for idx, item in enumerate(items_meta.index)}

In [10]:
from tqdm import tqdm

for epoch in range(EPOCHS):
    running_loss = 0.0
    # tqdm для прогресса по батчам
    with tqdm(train_dl, desc=f"Epoch {epoch+1}/{EPOCHS}", unit="batch") as t:
        for users, items, ages, item_durations, targets in t:
            # Извлекаем embeddings
            indices = [item_id_to_index[item.item()] for item in items]
            embeddings = item_embeddings_array[indices]
            
            # Объединяем признаки
            inputs = torch.cat((
                users.unsqueeze(1),
                items.unsqueeze(1),
                ages.unsqueeze(1),
                item_durations.unsqueeze(1),
                embeddings
            ), dim=1).float()
            
            # Остальной процесс обучения
            targets = targets
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.long())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            t.set_postfix(mean_loss=f"{running_loss / (t.n + 1):.6f}")


    
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_dl):.4f}")

Epoch 1/4: 100%|██████████| 8891/8891 [1:22:20<00:00,  1.80batch/s, mean_loss=0.200676]


Epoch [1/4], Loss: 0.2007


Epoch 2/4: 100%|██████████| 8891/8891 [1:21:44<00:00,  1.81batch/s, mean_loss=0.170801]


Epoch [2/4], Loss: 0.1708


Epoch 3/4: 100%|██████████| 8891/8891 [1:22:31<00:00,  1.80batch/s, mean_loss=0.169409]


Epoch [3/4], Loss: 0.1694


Epoch 4/4: 100%|██████████| 8891/8891 [1:22:57<00:00,  1.79batch/s, mean_loss=0.168941]

Epoch [4/4], Loss: 0.1689


In [11]:
torch.save(model.state_dict(), model_path)
print(f"Модель сохранена в {model_path}")

Модель сохранена в 2.1_DCN_MLP.pth
